In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import (
    aggregate,
    datashade,
    dynspread,
    shade,
    regrid,
)
from holoviews.operation import decimate
from holoviews.streams import Stream, param
import matplotlib.pyplot as plt
from matplotlib.colors import hex2color
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output
import nd2reader
from numcodecs import Blosc, Delta
import zarr
import skimage
import skimage.morphology
import sklearn
from sklearn.cluster import DBSCAN

# from sklearn import metrics
# from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import peakutils
import scipy.stats
import scipy.interpolate
import functools
from collections import Counter
from bokeh.models import WheelZoomTool
from bokeh.io import push_notebook, show, output_notebook

# from bokeh.layouts import row
# from bokeh.plotting import figure
from tqdm import tnrange, tqdm_notebook

In [ ]:
%load_ext line_profiler
hv.notebook_extension("bokeh")
renderer = hv.renderer("bokeh")
%matplotlib inline

In [ ]:
frames_z = zarr.open_array("/home/jqs1/scratch/fidelity/test/171018.zarr", mode="r")

In [ ]:
frames = nd2reader.ND2Reader(
    "/home/jqs1/scratch/fidelity/171018/20171018_TrxnError_ID.nd2"
)

In [ ]:
frames_z.shape

In [ ]:
channel_colors = {
    "BF": "#ffffff",
    "MCHERRY": "#e22400",
    "GFP": "#76ba40",
    "CY5": "#e292fe",
    "BFP": "#3a87fd",
    "YFP": "#f5eb00",
}

In [ ]:
%%output size=250

channels = frames_z.attrs["metadata"]["channels"]
n_channels = len(channels)
colors = [hex2color(channel_colors[channel]) for channel in channels]
num_timepoints = len(frames_z.attrs["metadata"]["frames"])
num_fovs = len(frames_z.attrs["metadata"]["fields_of_view"])

channel_boxes = []
channel_widgets = []
for channel in channels:
    solo_button = widgets.Button(description="S", layout=widgets.Layout(width="10%"))
    enabled_button = widgets.ToggleButton(description=channel, value=True)
    solo_button._button_to_enable = enabled_button
    color_picker = widgets.ColorPicker(concise=True, value=channel_colors[channel])
    channel_box = widgets.HBox([solo_button, enabled_button, color_picker])
    channel_widgets.append([solo_button, enabled_button, color_picker, channel_box])
solo_buttons, enabled_buttons, color_pickers, channel_boxes = zip(*channel_widgets)
channels_box = widgets.VBox(channel_boxes)
t_slider = widgets.IntSlider(
    label="t", min=0, max=num_timepoints, step=1, value=0, continuous_update=False
)
v_slider = widgets.IntSlider(
    min=0, max=num_fovs, step=1, value=0, continuous_update=False
)
slider_box = widgets.VBox([v_slider, t_slider])
control_box = widgets.HBox([channels_box, slider_box])
output = widgets.Output()
main_box = widgets.VBox([control_box, output])
display(main_box)

max_val = 2**14

Frame = Stream.define("Frame", t=0, v=0)
frame = Frame()
DisplaySettings = Stream.define(
    "DisplaySettings", channel_enabled=np.array([True] * n_channels)
)
display_settings = DisplaySettings()


def composite_image(t, v, channel_enabled):
    # def composite_image(t, v):
    # channel_enabled = [True] * n_channels
    # channel_imgs = [frames.get_frame_2D(c=i, t=t, v=v) for i in range(n_channels)]
    channel_imgs = [frames_z[v, c, t, :, :] for c in range(n_channels)]
    scaled_imgs = [
        channel_imgs[i][:, :, np.newaxis] / np.percentile(channel_imgs[i], 99.9)
        for i in range(n_channels)
    ]
    for scaled_img in scaled_imgs:
        np.clip(scaled_img, 0, 1, scaled_img)  # clip in place
    colored_imgs = [scaled_imgs[i] * np.array(colors[i]) for i in range(n_channels)]
    imgs_to_combine = [colored_imgs[i] for i in range(n_channels) if channel_enabled[i]]
    if not len(imgs_to_combine):
        imgs_to_combine = [np.ones(colored_imgs[0].shape)]  # white placeholder
    img = imgs_to_combine[0]
    for img2 in imgs_to_combine[1:]:
        img = 1 - (1 - img) * (1 - img2)
    return hv.RGB(img, bounds=(-1, -1, 1, 1))  # .opts(plot={'size': 250}, tools=[''])


t_slider.observe(lambda change: frame.event(t=change["new"]), names="value")
v_slider.observe(lambda change: frame.event(v=change["new"]), names="value")


def update_enabled_channels(change):
    channel_enabled = np.array([button.value for button in enabled_buttons])
    display_settings.event(channel_enabled=channel_enabled)


def update_solo(solo_button):
    if (
        solo_button._button_to_enable.value
        and sum([b.value for b in enabled_buttons]) == 1
    ):
        for enabled_button in enabled_buttons:
            enabled_button.value = True
    else:
        for enabled_button in enabled_buttons:
            enabled_button.value = enabled_button == solo_button._button_to_enable
    # update_enabled_channels(None)


for solo_button in solo_buttons:
    solo_button.on_click(update_solo)

for enabled_button in enabled_buttons:
    enabled_button.observe(update_enabled_channels, names="value")
# for color_picker in color_pickers:
#    color_picker.observe(update_image, names='value')

# hv.DynamicMap(composite_image, kdims=['t', 'v', 'channel_enabled']).select(t=0,v=0,channel_enabled=np.array([True,False,False,False,False]))
image_viewer = hv.DynamicMap(composite_image, streams=[frame, display_settings])
regrid(image_viewer)

In [ ]:
a_stack = frames_z[0, :, :, 300:500, :1000]
a = a_stack[0, 0]

In [ ]:
%%time
f0 = frames_z[0, 0, :].max(axis=0)

In [ ]:
f1 = f0 - np.percentile(f0, 0.5, axis=1)[:, np.newaxis]

In [ ]:
f1_b = skimage.filters.gaussian(f1, 3)
f1_v = skimage.filters.sobel_v(f1_b)
f1_v2 = skimage.filters.sobel_v(f1_v)

In [ ]:
def hessian_eigenvalues(img):
    I = skimage.filters.gaussian(img, 1.5)
    I_x = skimage.filters.sobel_h(I)
    I_y = skimage.filters.sobel_v(I)
    I_xx = skimage.filters.sobel_h(I_x)
    I_xy = skimage.filters.sobel_v(I_x)
    I_yx = skimage.filters.sobel_h(I_y)
    I_yy = skimage.filters.sobel_v(I_y)
    kappa_1 = (I_xx + I_yy) / 2
    kappa_2 = (np.sqrt((I_xx + I_yy) ** 2 - 4 * (I_xx * I_yy - I_xy * I_yx))) / 2
    k1 = kappa_1 + kappa_2
    k2 = kappa_1 - kappa_2
    k1[np.isnan(k1)] = 0
    k2[np.isnan(k2)] = 0
    return k1, k2

In [ ]:
f1_k1, f1_k2 = hessian_eigenvalues(f1_b)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(f1)

# Trench detection

In [ ]:
def cluster_binary_image(bin_img):
    X = np.array(np.where(bin_img)).T
    X2 = StandardScaler().fit_transform(X.astype(np.float32))
    fit = sklearn.cluster.MiniBatchKMeans(
        init="k-means++", n_clusters=2, n_init=10, max_no_improvement=10, verbose=0
    )
    fit.fit(X2)
    return X, fit


def label_binary_image(bin_img):
    X, fit = cluster_binary_image(bin_img)
    label_img = np.zeros_like(bin_img)
    for i in range(len(fit.labels_)):
        label_img[X[i, 0], X[i, 1]] = fit.labels_[i] + 1
    return label_img


def drop_rare_labels(labels):
    total = fit.labels_.sum()
    good_labels = []
    for label, count in enumerate(fit.labels_):
        print(count / total)
        if count / total > 0.01:
            good_labels.append(label)
    return good_labels


def detect_trenches(bin_img):
    h, theta, d = skimage.transform.hough_line(bin_img)
    abs_diff_h = np.diff(h.astype(np.int32), axis=1).var(axis=0)
    theta_idx = abs_diff_h.argmax()
    angle1 = theta[theta_idx]
    h2, theta2, d2 = skimage.transform.hough_line(
        bin_img, theta=np.linspace(0.9 * angle1, 1.1 * angle1, 200)
    )
    abs_diff_h2 = np.diff(h2.astype(np.int32), axis=1).var(axis=0)
    theta_idx2 = abs_diff_h2.argmax()
    angle2 = theta2[theta_idx2]
    d_profile = h2[:, theta_idx2].astype(np.int32)
    freqs = np.abs(np.fft.fft(d_profile))
    peak_idxs = peakutils.indexes(d_profile, thres=0.4, min_dist=5)
    peaks = d2[peak_idxs]
    spacing = scipy.stats.mode(np.diff(peaks)).mode[0]
    return angle2, peaks


def get_rough_spacing(dists):
    spacing = scipy.stats.mode(np.diff(dists).astype(int)).mode[0]
    return spacing


def point_linspace(anchor0, anchor1, num_points):
    for s in np.linspace(0, 1, num_points)[1:-1]:
        anchor = (1 - s) * anchor0 + s * anchor1
        yield anchor


def coords_along(x0, x1):
    length = int(np.sqrt(np.sum((x1 - x0) ** 2)))
    xs = np.linspace(x0[0], x1[0], length).astype(np.int_)[1:-1]
    ys = np.linspace(x0[1], x1[1], length).astype(np.int_)[1:-1]
    return xs, ys


def edge_point2(x0, theta, x_lim, y_lim):
    x_min, x_max = x_lim
    y_min, y_max = y_lim
    if 0 <= theta < np.pi / 4 or 5 / 4 * np.pi <= 2 * np.pi:
        x, y = x_min, y_min
        angle_to_corner = np.arctan2(-x0[0], x0[1])
    else:
        x, y = x_max, y_max
        angle_to_corner = -np.arctan2(x0[0] - x_max, -(y_max - x0[1]))
    angle_to_corner = angle_to_corner % (2 * np.pi)
    print("corner angle", angle_to_corner, theta)
    if angle_to_corner < theta:
        # endpoint at top/bottom
        x1 = np.array([x0[0] + (x0[1] - y) * np.tan(theta), y])
    else:
        # endpoint on side
        x1 = np.array([x, x0[1] + (x0[0] - x) / np.tan(theta)])
    return x1

In [ ]:
np.pi

In [ ]:
(-np.pi / 4) % (2 * np.pi)

In [ ]:
np.arctan2(-1, 1) % (np.pi * 2)

In [ ]:
def edge_point2(x0, theta, x_lim, y_lim):
    x_min, x_max = x_lim
    y_min, y_max = y_lim
    theta = theta % (2 * np.pi)
    if 0 <= theta < np.pi / 2:
        x, y = x_min, y_max
        angle_to_corner = np.arctan2(y_max - x0[1], x0[0] - x_min)
        # print('atc', angle_to_corner)
        if angle_to_corner <= theta:
            # endpoint at top/bottom
            x1 = np.array([x0[0] + (x0[1] - y) / np.tan(theta), y])
            # x1 = np.array([0,0])
        else:
            # endpoint on side
            x1 = np.array([x, x0[1] + (x0[0] - x) * np.tan(theta)])
    elif np.pi / 2 <= theta < np.pi:
        x, y = x_max, y_max
        angle_to_corner = np.arctan2(y_max - x0[1], x0[0] - x_max)
        print("atc", angle_to_corner)
        if angle_to_corner > theta:
            # endpoint at top/bottom
            x1 = np.array([x0[0] + (x0[1] - y) / np.tan(theta), y])
        else:
            # endpoint on side
            x1 = np.array([x, x0[1] + (x0[0] - x) * np.tan(theta)])
    elif np.pi <= theta < 3 / 2 * np.pi:
        x, y = x_max, y_min
        angle_to_corner = np.arctan2(y_min - x0[1], x0[0] - x_max) % (2 * np.pi)
        # print('atc', angle_to_corner)
        if angle_to_corner <= theta:
            # endpoint at top/bottom
            x1 = np.array([x0[0] + (x0[1] - y) * np.tan(theta), y])
        else:
            # endpoint on side
            # x1 = np.array([x, x0[1] + (x0[0] - x)/np.tan(theta-np.pi)])
            # print(np.tan(theta), x0[0] - x)
            x1 = np.array([x, x0[1] + (x0[0] - x) * np.tan(theta)])
    elif 3 / 2 * np.pi <= theta <= 2 * np.pi:
        x, y = x_min, y_min
        angle_to_corner = np.arctan2(y_min - x0[1], -(x_min - x0[0])) % (2 * np.pi)
        print("atc", angle_to_corner)
        if angle_to_corner <= theta:
            # endpoint on side
            # x1 = np.array([x0[0] + (x0[1] - y)/np.tan(theta), y])
            print(np.tan(theta), x0[1] - y)
            # x1 = np.array([0,0])
            x1 = np.array([x, x0[1] + (x0[1] - y) * np.tan(theta)])
        else:
            # endpoint at top/bottom
            # x1 = np.array([x, x0[1] + (x0[0] - x)*np.tan(theta)])
            x1 = np.array([0, 0])
    return x1

In [ ]:
def edge_point(x0, theta, x_lim, y_lim):
    x_min, x_max = x_lim
    y_min, y_max = y_lim
    theta = theta % (2 * np.pi)
    if 0 <= theta < np.pi / 2:
        corner_x, corner_y = x_min, y_max
    elif np.pi / 2 <= theta < np.pi:
        corner_x, corner_y = x_max, y_max
    elif np.pi <= theta < 3 / 2 * np.pi:
        corner_x, corner_y = x_max, y_min
    elif 3 / 2 * np.pi <= theta <= 2 * np.pi:
        corner_x, corner_y = x_min, y_min
    # angle_to_corner = np.arctan2(corner_y-x0[1], x0[0]-corner_x) % (2*np.pi)
    angle_to_corner = np.arctan2(corner_y - x0[1], x0[0] - corner_x) % (2 * np.pi)
    # print('atc', angle_to_corner*180/np.pi)
    # print('$$', (corner_y-x0[1], x0[0]-corner_x))
    if (
        (theta >= angle_to_corner and 0 <= theta < np.pi / 2)
        or (theta < angle_to_corner and np.pi / 2 <= theta < np.pi)
        or (theta >= angle_to_corner and np.pi <= theta < 3 / 2 * np.pi)
        or (theta < angle_to_corner and 3 / 2 * np.pi <= theta < 2 * np.pi)
    ):
        # top/bottom
        # print(theta, np.tan(theta), abs(corner_y - x0[1])/np.tan(theta))
        x1 = np.array([x0[0] - (corner_y - x0[1]) / np.tan(theta), corner_y])
    else:
        # left/right
        # x1 = x0
        x1 = np.array([corner_x, x0[1] - (corner_x - x0[0]) * np.tan(theta)])
    return x1


# x0 = np.array([1024,1024])
# x0 = np.array([524,1024])
x0 = np.array([500, 124])
plt.figure(figsize=(12, 12))
plt.gca().add_artist(plt.Circle(x0, 10, color="g", zorder=2))
for i, theta in enumerate(np.linspace(0, 2 * np.pi, 50)):
    # for i, theta in enumerate(np.linspace(3/2*np.pi, 4/2*np.pi, 50)):
    # for i, theta in enumerate([0, np.pi/4-.01, np.pi/4+.01]):
    x1 = edge_point(x0, theta, x_lim, y_lim)
    # print('>>', theta*180/np.pi, x1)
    line = np.vstack((x0, x1)).T
    plt.plot(*line, color="gray")
    plt.gca().add_artist(plt.Circle(x1, 10, color="r", zorder=2))
    # plt.gca().text(x1[0], x1[1], 'i:{}'.format(i), fontsize=15)
    # plt.annotate(str(i), xy=tuple(x1), textcoords='data')
plt.xlim(x_lim)
plt.ylim(y_lim)

In [ ]:
(np.arctan2(-500, -124) * 180 / np.pi) % 360

In [ ]:
(np.arctan2(-124, 500) * 180 / np.pi) % 360

In [ ]:
def line_array(
    anchors, theta, x_lim, y_lim, start=None, stop=None, bidirectional=False
):
    if bidirectional:
        line_array1 = line_array(
            anchors, theta, x_lim, y_lim, start=start, stop=stop, bidirectional=False
        )
        line_array2 = line_array(
            anchors,
            theta + np.pi,
            x_lim,
            y_lim,
            start=start,
            stop=stop,
            bidirectional=False,
        )
        for (x0, x1), (y0, y1) in zip(line_array1, line_array2):
            yield x1, y1
        return
    if start is None:
        start = 0
    if stop is None:
        stop = 0
    if not stop >= start >= 0:
        raise ValueError("need stop >= start >= 0")
    # TODO: standardize 0 < theta < 2pi
    print("theta", theta)
    theta = theta % (2 * np.pi)
    print("newtheta", theta)
    for anchor in anchors:
        x0 = anchor
        x1 = edge_point(x0, theta, x_lim, y_lim)
        # max_length = np.sqrt(((x1 - x0)**2).sum())
        y0, y1 = x0, x1
        # if start:
        #    y0 = min(start / max_length, 1) * (x1 - x0) + x0
        # if stop:
        #    y1 = min(stop / max_length, 1) * (x1 - x0) + x0
        if not np.array_equal(y0, y1):
            yield y0, y1


def point_linspace_fine(anchor0, anchor1, spacing, offset):
    for s in np.linspace(0, 1, N)[1:-1]:
        anchor = (1 - s) * anchor0 + s * anchor1
        yield anchor


def refine_trenches(img, theta, spacing):
    x_min = y_min = 0
    x_max, y_max = img.shape
    x_lim = (x_min, x_max)
    y_lim = (y_min, y_max)
    anchor0, anchor1 = get_anchors(theta, x_lim, y_lim)

    def objective(spacing_, offset):
        s = 0
        for x0, x1 in line_array(
            point_linspace_fine(anchor0, anchor1, spacing_, offset), theta, x_lim, y_lim
        ):
            xs, ys = coords_along(x0, x1)
            s += img[ys, xs].sum()
        return s

    spacings = np.linspace(0.8 * spacing, 1.2 * spacing, 20)
    offsets = np.linspace(0, 1.2 * spacing, 20)


def get_anchors(theta, x_lim, y_lim):
    x_min, x_max = x_lim
    y_min, y_max = y_lim
    if 0 <= (theta % np.pi) < np.pi / 4 or 3 / 4 * np.pi <= (theta % np.pi) < np.pi:
        y0 = (y_max - y_min) / 2 + y_min
        dy = (x_max - x_min) / 2 * np.tan(theta)
        anchor0 = np.array([x_min, y0 - dy])
        anchor1 = np.array([x_max, y0 + dy])
    else:
        x0 = (x_max - x_min) / 2 + x_min
        dx = (y_max - y_min) / 2 / np.tan(theta)
        anchor0 = np.array([x0 - dx, y_min])
        anchor1 = np.array([x0 + dx, y_max])
    if 3 / 4 * np.pi <= (theta % (2 * np.pi)) < 7 / 4 * np.pi:
        anchor0, anchor1 = anchor1, anchor0
    return anchor0, anchor1


# img = skimage.transform.rotate(f1, 15, cval=0)
# img_thresh = img > skimage.filters.threshold_otsu(img)
# img_labels = label_binary_image(img_thresh)
# theta, dists = detect_trenches(img_labels == 1)
# spacing = get_rough_spacing(dists)
##########
# anchors
x_min = y_min = 0
x_max, y_max = img.shape
x_lim = (x_min, x_max)
y_lim = (y_min, y_max)
theta2 = 0  # np.pi/2#np.pi/2 + theta
anchor0, anchor1 = get_anchors(theta2, x_lim, y_lim)
plt.figure(figsize=(12, 12))
plt.imshow(img)
plt.gca().add_artist(plt.Circle(anchor0, 50, color="g"))
plt.gca().add_artist(plt.Circle(anchor1, 50, color="gray"))
trench_profiles = []
# lines = line_array(point_linspace(anchor0, anchor1, int((anchor1[0] - anchor0[0])//spacing)), theta, x_lim, y_lim, start=500, stop=700)
lines = line_array(
    point_linspace(anchor0, anchor1, int((anchor1[0] - anchor0[0]) // spacing)),
    3 * np.pi / 2,
    x_lim,
    y_lim,
    bidirectional=False,
)
for x0, x1 in lines:
    print(x0, x1)
    xs, ys = coords_along(x0, x1)
    trench_profiles.append(img[ys, xs])
    line = np.vstack((x0, x1)).T
    plt.plot(*line, color="w")
    plt.gca().add_artist(plt.Circle(x0, 10, color="r", zorder=2))
    plt.gca().add_artist(plt.Circle(x1, 10, color="r", zorder=2))
plt.figure(figsize=(12, 12))
for trench_profile in trench_profiles:
    plt.plot(trench_profile)